In [381]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
#NOTE: Will need to re-run this after scraping and saving an updated year (2025) in get_data (updated as of 1/28/24 games on 1/29) 

In [376]:
SCORES_DIR = "data/scores"
CSV_FILE_NAME = "nba_games.csv"

In [377]:
box_scores = os.listdir(SCORES_DIR)

In [378]:
box_scores = [os.path.join(SCORES_DIR, f) for f in box_scores if f.endswith(".html")] #join together full path to scores
box_scores[0]

'data/scores/201510270ATL.html'

In [379]:
saved_df = pd.DataFrame()
#Check if we can append to an existing csv file to save time
if os.path.exists(CSV_FILE_NAME):
    saved_df = pd.read_csv(CSV_FILE_NAME, parse_dates=["date"])     
    last_game_date = saved_df["date"].max()  
    print(f"The last saved game was played on {last_game_date}")
    #Create set for already processed games to prevent duplicates of team playing same game
    saved_games = set(zip(saved_df["date"], saved_df["team"]))
    #Now, we can omit box scores that we've already processed
    new_box_scores = []
    for box_score in box_scores:
        # Grab yyyymmdd from filename and format into timestamp to compare
        file_date = os.path.basename(box_scores[0])[:8] 
        file_date = pd.to_datetime(os.path.basename(box_score)[:8], format="%Y%m%d")
        #Only add if file box score is after most recent game, could skip games if you processed one game on a day and not the rest but unlikely
        if file_date > last_game_date: 
            new_box_scores.append(box_score)
    box_scores = new_box_scores
    print(f"Will process {len(box_scores)} new games...") #Note that each file is a game that will output two box score rows

The last saved game was played on 2025-01-28 00:00:00
Will process 0 new games...


In [380]:
saved_df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24423,240.0,36.0,92.0,0.391,17.0,49.0,0.347,25.0,31.0,0.806,...,30.7,28.6,200.0,127.0,UTA,103,0,2025,2025-01-28,True
24424,240.0,37.0,72.0,0.514,10.0,33.0,0.303,20.0,23.0,0.870,...,33.3,37.2,162.0,115.0,PHI,118,1,2025,2025-01-28,False
24425,240.0,43.0,91.0,0.473,11.0,34.0,0.324,21.0,25.0,0.840,...,37.5,40.7,227.0,128.0,LAL,104,0,2025,2025-01-28,True
24426,240.0,44.0,87.0,0.506,11.0,35.0,0.314,13.0,18.0,0.722,...,33.3,35.2,250.0,123.0,POR,125,1,2025,2025-01-28,False


In [361]:
def parse_html(box_score):
    try:
        with open(box_score, encoding = 'utf-8') as f:
            html = f.read()
        soup = BeautifulSoup(html)
        #Remove the "Basic Box Score Stats" header that would result in a multi-layered header
        [s.decompose() for s in soup.select("tr.over_header")]
        #Remove the repeated header for Reserves/MP/FG etc. that is halfway through the table
        [s.decompose() for s in soup.select("tr.thead")]
        return soup;
    except Exception as e:
        print(f"Error occurred while processing file: {box_score}")
        print(f"Error: {e}")
        #raise
        return None # Continue looping but signify to caller that error happened 

In [362]:
#Line score is already in an html table, so we can just feed it directly into pandas
def read_line_score(soup):
    #Grab table with id line_score, then access first element since read_html returns list by default
    line_score = pd.read_html(StringIO(str(soup)), attrs={"id":"line_score"})[0] #str(soup) converts HTML page to string to read
    cols = list(line_score.columns)
    #Rename first column and last
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    #In between team/total columns there are quarterly columns that cause issue when there is overtime, so only include the 2
    line_score = line_score[["team", "total"]]
    return line_score

In [363]:
def read_stats(soup, team, table_type):
    box_score_id = f"box-{team}-game-{table_type}" #table_type is basic or advanced stats
    #Grab html table using pandas, indexing by the player column
    box_score_df = pd.read_html(StringIO(str(soup)), attrs={"id":box_score_id}, index_col = 0)[0]
    #Convert all columns to numeric, if it cannot then NAN with coerce arg
    box_score_df = box_score_df.apply(pd.to_numeric, errors="coerce")
    return box_score_df    

In [364]:
def read_season_info(soup):
    #Select bottom nav div by its id (and index since it always returns list)
    nav_div = soup.select("#bottom_nav_container")[0]
    #Get all the hrefs in the a tags into an array
    a_tags = nav_div.find_all("a")
    hrefs = [l.get("href") for l in a_tags] 
    #Ignore any a tags without html 
    hrefs = [l for l in hrefs if ".html" in l]
    #Grab the basename of the URL ({year}_games.html), split by the underscore, and grab first element (the year)
    season_year = os.path.basename(hrefs[0]).split("_")[0]
    return season_year

In [365]:
base_cols = None
games = []

#for i in range(10600, len(box_scores)):
for box_score in box_scores:    
    soup = parse_html(box_score)
    if soup is None:
        print(f"Skipping {box_score} due to error in parsing html")
        continue #Continue to next file
        
    line_score = read_line_score(soup)
    teams = list(line_score["team"]) #Grab the team abbreviations
    
    summaries = []
    for team in teams:
        #Get the basic/advanced tables as dataframes for each team
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
        #Get last row of each table (the team totals)
        basic_total_row = basic.iloc[-1]
        advanced_total_row = advanced.iloc[-1]
        #Then, concatenate into one series
        totals = pd.concat([basic_total_row, advanced_total_row])
        totals.index  = totals.index.str.lower()
        #Now, to compress the rows of all the players, we'll get the player rows (so [0:-1]) where a max stat was achieved 
        maxes = pd.concat([basic.iloc[0:-1].max(), advanced.iloc[0:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max" #Distinguish from totals by appending max
        #Combine totals and maxes into one series
        summary = pd.concat([totals, maxes])
        #Different box scores sometimes have extra box_scores
        #So, for our first iteration, we will standardize by including ONLY the base_cols (and drop dupes/extra stat @ same time)
        if base_cols is None: # Note that if we loaded in previous CSV data, we instead use that as base columns
            base_cols = list(summary.index.drop_duplicates(keep="first")) #Drop duplicate stats like MP that are the same
            base_cols = [b for b in base_cols if "bpm" not in b] #Remove bpm stat 
        summary = summary[base_cols]
        #Save summary for each of the two teams
        summaries.append(summary) 
    summary = pd.concat(summaries, axis=1).T #Transpose
    #Combine summary that has totals/maxes, with line score as well (axis=1 to add new columns)
    game = pd.concat([summary, line_score], axis=1)
    #Assign new column for home or away (first team is always away in html, so can make it hard-coded)
    game["home"] = [0,1]
    #Create reversed dataframe that has home first, away second, and rename columns to signify 
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"
    
    #Now, we concatenate back, so that we have opponent stats for each team in each row (i.e. home team has opp stats in their row)
    full_game = pd.concat([game, game_opp], axis=1)
    #Add context for season and date (latter of which we can grab from filename)
    full_game["season"] = read_season_info(soup)
    full_game["date"] = os.path.basename(box_score)[:8] #First 8 characters is year, month, day, so then convert to nicer format
    full_game["date"] = pd.to_datetime(full_game["date"], format = "%Y%m%d")
    full_game["won"] = full_game["total"] >full_game["total_opp"]
    #Save game to list
    games.append(full_game)
    #Report progress every 100 games
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

In [375]:
#print(list(games[14].columns))

In [369]:
#Concatenate all games together
games_df = pd.concat(games, ignore_index = True)
#[g.shape[1] for g in games if g.shape[1] != 150]  #Check error for df structure
#[g.shape for g in games]

In [370]:
#Delete extra columns that were duplicated (mp, mp_opp, mp_max, mp_max_opp) by selecting only the non-duplicated flag bools
games_df = games_df.loc[:,~games_df.columns.duplicated()].copy()
#Convert seasons from string to int
games_df['season'] = pd.to_numeric(games_df['season'], errors='coerce')

In [371]:
games_df.columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'tov%_max_opp', 'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp',
       'team_opp', 'total_opp', 'home_opp', 'season', 'date', 'won'],
      dtype='object', length=150)

In [372]:
#Check if we had previous data that we have to concatenate with new data
if not saved_df.empty:
    #Make sure both dataframes have the same columns by dropping columns that are not in saved_df (this'll likely drop columns that would've been removed by base_cols normally like gmsc)
    common_columns = saved_df.columns.intersection(games_df.columns)
    games_df = games_df[common_columns]
    #Finally, concatenate our old data to our new data
    games_df = pd.concat([saved_df, games_df], ignore_index = True)
    print("Old box scores have been merged with new ones")

Old box scores have been merged with new ones


In [373]:
games_df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24423,240.0,36.0,92.0,0.391,17.0,49.0,0.347,25.0,31.0,0.806,...,30.7,28.6,200.0,127.0,UTA,103,0,2025,2025-01-28,True
24424,240.0,37.0,72.0,0.514,10.0,33.0,0.303,20.0,23.0,0.870,...,33.3,37.2,162.0,115.0,PHI,118,1,2025,2025-01-28,False
24425,240.0,43.0,91.0,0.473,11.0,34.0,0.324,21.0,25.0,0.840,...,37.5,40.7,227.0,128.0,LAL,104,0,2025,2025-01-28,True
24426,240.0,44.0,87.0,0.506,11.0,35.0,0.314,13.0,18.0,0.722,...,33.3,35.2,250.0,123.0,POR,125,1,2025,2025-01-28,False


In [374]:
games_df.to_csv(CSV_FILE_NAME, index=False)

In [ ]:
games_df.compare(saved_df)

In [347]:
saved_df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240,37,96,0.385,12,29,0.414,20,26,0.769,...,57.1,33.8,258,121,ATL,94,1,2016,2015-10-27,True
1,240,37,82,0.451,8,27,0.296,12,15,0.800,...,33.3,23.6,132,104,DET,106,0,2016,2015-10-27,False
2,240,38,94,0.404,9,29,0.310,10,17,0.588,...,53.2,34.6,162,104,CHI,97,1,2016,2015-10-27,False
3,240,37,87,0.425,7,19,0.368,16,23,0.696,...,30.4,29.0,138,105,CLE,95,0,2016,2015-10-27,True
4,240,35,83,0.422,6,18,0.333,19,27,0.704,...,69.4,43.7,206,104,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24391,240,40,90,0.444,13,40,0.325,26,28,0.929,...,37.5,38.4,147,133,WAS,109,0,2025,2025-01-25,True
24392,240,53,102,0.520,17,39,0.436,13,14,0.929,...,100.0,35.2,152,145,SAS,98,1,2025,2025-01-25,True
24393,240,35,85,0.412,11,32,0.344,17,25,0.680,...,25.0,28.7,250,111,IND,136,0,2025,2025-01-25,False
24394,240,42,89,0.472,10,30,0.333,24,29,0.828,...,29.1,34.7,154,134,POR,108,1,2025,2025-01-26,True


In [348]:
games_df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24423,240.0,36.0,92.0,0.391,17.0,49.0,0.347,25.0,31.0,0.806,...,30.7,28.6,200.0,127.0,UTA,103,0,2025,2025-01-28,True
24424,240.0,37.0,72.0,0.514,10.0,33.0,0.303,20.0,23.0,0.870,...,33.3,37.2,162.0,115.0,PHI,118,1,2025,2025-01-28,False
24425,240.0,43.0,91.0,0.473,11.0,34.0,0.324,21.0,25.0,0.840,...,37.5,40.7,227.0,128.0,LAL,104,0,2025,2025-01-28,True
24426,240.0,44.0,87.0,0.506,11.0,35.0,0.314,13.0,18.0,0.722,...,33.3,35.2,250.0,123.0,POR,125,1,2025,2025-01-28,False
